In [27]:
import sys
import plotly
import cvxpy as cvx
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import yfinance as yf
import requests
from bs4 import BeautifulSoup
from sklearn.decomposition import PCA

%matplotlib inline
plt.style.use('ggplot')
plt.rcParams['figure.figsize'] = (14, 8)

In [29]:
# Function to fetch the list of S&P 500 companies from Wikipedia
def get_sp500_tickers():
    url = 'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies'
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    table = soup.find('table', {'id': 'constituents'})
    tickers = []
    for row in table.find_all('tr')[1:]:
        ticker = row.find_all('td')[0].text.strip().replace('.', '-')
        tickers.append(ticker)
    return tickers

# Fetch the list of S&P 500 tickers
sp500_tickers = get_sp500_tickers()

In [30]:
# Function to fetch stock data for a list of tickers
def fetch_stock_data(tickers, start_date, end_date):
    all_data = []
    problematic_tickers = []
    for ticker in tickers:
        try:
            stock = yf.Ticker(ticker)
            hist = stock.history(start=start_date, end=end_date)
            if not hist.empty:
                hist.reset_index(inplace=True)
                hist['Ticker'] = ticker
                all_data.append(hist)
            else:
                print(f"No data found for {ticker}")
                problematic_tickers.append(ticker)
        except Exception as e:
            print(f"Could not download data for {ticker}: {e}")
            problematic_tickers.append(ticker)
    if all_data:
        return pd.concat(all_data, ignore_index=True), problematic_tickers
    else:
        return pd.DataFrame(), problematic_tickers

In [31]:
# Fetch data for the desired date range
start_date = '2023-01-01'
end_date = '2023-12-31'
stock_data, problematic_tickers = fetch_stock_data(sp500_tickers, start_date, end_date)

if not stock_data.empty:
    # Format the DataFrame
    stock_data = stock_data[['Ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    stock_data['Adj Close'] = stock_data['Close']  # Assuming 'Adj Close' is the same as 'Close' for simplicity
    stock_data['Adj Volume'] = stock_data['Volume']  # Assuming 'Adj Volume' is the same as 'Volume' for simplicity

    # Save to CSV
    file_path = 'sp500_stock_prices.csv'
    stock_data.to_csv(file_path, index=False)

    print("CSV file saved at:", file_path)
else:
    print("No stock data available to save.")

if problematic_tickers:
    print("Problematic tickers:", problematic_tickers)

GEV: Data doesn't exist for startDate = 1672549200, endDate = 1703998800


No data found for GEV


SW: Data doesn't exist for startDate = 1672549200, endDate = 1703998800


No data found for SW


SOLV: Data doesn't exist for startDate = 1672549200, endDate = 1703998800


No data found for SOLV
CSV file saved at: sp500_stock_prices.csv
Problematic tickers: ['GEV', 'SW', 'SOLV']


In [34]:
stock_data

,Ticker,Date,Open,High,Low,Close,Volume,Adj Close,Adj Volume
0,MMM,2023-01-03 00:00:00-05:00,93.597492,94.460141,92.711734,94.329201,3124909,94.329201,3124909
1,MMM,2023-01-04 00:00:00-05:00,95.006997,96.501227,94.514053,96.393394,3312561,96.393394,3312561
2,MMM,2023-01-05 00:00:00-05:00,95.669391,95.946670,94.321500,94.706612,3117494,94.706612,3117494
3,MMM,2023-01-06 00:00:00-05:00,96.015990,97.918447,95.315092,97.602654,2890732,97.602654,2890732
4,MMM,2023-01-09 00:00:00-05:00,97.818299,99.713045,97.132800,97.656548,3434075,97.656548,3434075
...,...,...,...,...,...,...,...,...,...
124722,ZTS,2023-12-22 00:00:00-05:00,193.862804,194.448399,191.302051,193.525330,1548400,193.525330,1548400
124723,ZTS,2023-12-26 00:00:00-05:00,193.426089,194.875188,192.641974,194.041458,814600,194.041458,814600
124724,ZTS,2023-12-27 00:00:00-05:00,193.952126,195.540180,193.287126,195.431000,766400,195.431000,766400
124725,ZTS,2023-12-28 00:00:00-05:00,196.145631,197.118331,195.063767,195.689072,880100,195.689072,880100


In [45]:
#function to get mean for each stock
stock_data.groupby('Ticker').median()

,Date,Open,High,Low,Close,Volume,Adj Close,Adj Volume
Ticker,,,,,,,,
A,2023-07-04 00:00:00-04:00,125.843010,127.077878,124.344244,126.367733,1679150.0,126.367733,1679150.0
AAL,2023-07-04 00:00:00-04:00,14.380000,14.590000,14.220000,14.355000,24674600.0,14.355000,24674600.0
AAPL,2023-07-04 00:00:00-04:00,174.554627,176.327614,173.155161,174.783730,55077500.0,174.783730,55077500.0
ABBV,2023-07-04 00:00:00-04:00,142.405466,143.789490,140.886649,142.255981,4848250.0,142.255981,4848250.0
ABNB,2023-07-04 00:00:00-04:00,124.330002,126.660000,122.782501,124.994999,5058050.0,124.994999,5058050.0
...,...,...,...,...,...,...,...,...
XYL,2023-07-04 00:00:00-04:00,102.170872,102.963133,101.303393,102.202343,1346200.0,102.202343,1346200.0
YUM,2023-07-04 00:00:00-04:00,127.011918,127.835493,126.054801,127.003639,1454150.0,127.003639,1454150.0
ZBH,2023-07-04 00:00:00-04:00,123.673408,124.636950,122.587249,123.827843,1349700.0,123.827843,1349700.0


In [36]:
open_prices = stock_data.pivot(index='Date', columns='Ticker', values='Open')
high_prices = stock_data.pivot(index='Date', columns='Ticker', values='High')
low_prices = stock_data.pivot(index='Date', columns='Ticker', values='Low')
close_prices = stock_data.pivot(index='Date', columns='Ticker', values='Close')
volume = stock_data.pivot(index='Date', columns='Ticker', values='Volume')
adj_close_prices = stock_data.pivot(index='Date', columns='Ticker', values='Adj Close')
adj_volume = stock_data.pivot(index='Date', columns='Ticker', values='Adj Volume')

In [37]:
close_prices.shift(2)

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-05 00:00:00-05:00,148.440628,12.74,124.048050,151.773132,84.900002,105.825310,62.459999,262.890076,336.920013,158.037888,...,240.008942,28.833950,84.524162,66.355103,101.223030,108.735489,123.388397,125.783302,260.200012,144.436859
2023-01-06 00:00:00-05:00,150.053238,13.59,125.327507,152.997589,88.720001,107.399460,62.770000,261.995148,341.410004,161.403946,...,241.291351,29.179657,89.747078,66.923294,101.517639,108.568604,125.060005,126.949440,262.989990,146.512207
2023-01-09 00:00:00-05:00,150.488525,13.99,123.998444,152.810638,87.709999,107.003502,62.939999,255.808624,328.440002,155.352875,...,241.242401,28.310715,90.181496,65.569107,103.789009,105.485992,124.107574,124.597420,260.059998,143.059875
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,137.467819,13.98,194.318146,147.509125,141.070007,105.794914,73.610001,335.488770,596.059998,190.530746,...,233.160172,33.531780,88.113197,59.544312,99.529076,109.065689,127.769547,119.271240,261.609985,193.177948
2023-12-26 00:00:00-05:00,138.223907,14.35,194.168518,148.384323,142.089996,106.858772,74.160004,344.629486,600.140015,194.585434,...,235.961823,33.541618,90.272499,59.757980,99.981178,111.074562,128.700302,118.923355,265.309998,193.207733
2023-12-27 00:00:00-05:00,138.850677,14.31,193.091385,150.669540,140.800003,107.400558,73.839996,350.262177,598.750000,195.398361,...,237.684372,33.787453,89.973976,60.272720,100.158081,112.019325,128.720108,119.827827,269.410004,193.525330


In [12]:
def calculate_returns(close):
    """
    Compute returns for each ticker and date in close.
    
    Parameters
    ----------
    close : DataFrame
        Close prices for each ticker and date
    
    Returns
    -------
    returns : DataFrame
        Returns for each ticker and date
    """
    # TODO: Implement Function
    
    return (close - close.shift(1))/close.shift(1)

In [13]:
calculate_returns(close_prices)

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04 00:00:00-05:00,0.010864,0.066719,0.010314,0.008067,0.044994,0.014875,0.004963,-0.003404,0.013327,0.021299,...,0.005343,0.011990,0.061792,0.008563,0.002911,-0.001535,0.013548,0.009271,0.010722,0.014368
2023-01-05 00:00:00-05:00,0.002901,0.029433,-0.010605,-0.001222,-0.011384,-0.003687,0.002708,-0.023613,-0.037990,-0.037490,...,-0.000203,-0.029779,0.004840,-0.020235,0.022374,-0.028393,-0.007616,-0.018527,-0.011141,-0.023563
2023-01-06 00:00:00-05:00,-0.029189,0.013581,0.036794,0.018717,0.009235,0.013809,0.015253,0.023690,0.013123,0.036508,...,0.028364,0.043234,0.024414,0.029607,0.012087,0.035551,0.020282,-0.003014,0.051373,0.015057
2023-01-09 00:00:00-05:00,-0.001354,0.030324,0.004089,-0.029360,0.008134,-0.001602,-0.019249,0.016864,0.027739,0.009546,...,0.006432,-0.006011,0.008122,0.009679,-0.018638,0.004673,-0.004068,0.002784,0.021469,-0.003928
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,0.004534,-0.002787,-0.005547,0.015401,-0.009079,0.005070,-0.004315,0.016344,-0.002316,0.004178,...,0.007300,0.007329,-0.003307,0.008614,0.001769,0.008506,0.000154,0.007606,0.015454,0.001644
2023-12-26 00:00:00-05:00,0.001720,-0.013976,-0.002841,-0.002065,-0.014773,0.001834,-0.004469,-0.002878,-0.000818,0.008980,...,-0.004165,0.007858,0.006415,0.003062,0.002257,0.008700,0.002000,0.009142,0.022605,0.002667
2023-12-27 00:00:00-05:00,0.000072,-0.008505,0.000518,0.001682,-0.015643,0.005127,0.000408,0.001104,-0.003644,0.002414,...,0.000878,0.003754,-0.004066,-0.003078,-0.004699,0.006689,0.004453,-0.002388,0.001053,0.007161


In [148]:
close = stock_data.reset_index().pivot(index='Date', columns='Ticker', values='Adj Close')

In [154]:
def resample_prices(close_prices, freq='ME'):
    """
    Resample close prices for each ticker at specified frequency.
    
    Parameters
    ----------
    close_prices : DataFrame
        Close prices for each ticker and date
    freq : str
        What frequency to sample at
        For valid freq choices, see http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
    
    Returns
    -------
    prices_resampled : DataFrame
        Resampled prices for each ticker and date
    """
    # TODO: Implement Function

    return close_prices.resample(freq).last()

In [156]:
resample_prices(close)

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-31 00:00:00-05:00,150.458878,16.139999,143.110992,139.411575,111.110001,107.245453,64.349998,272.550690,370.339996,166.812958,...,248.828949,32.169567,102.325485,65.124023,110.251472,102.108879,126.838516,125.842598,316.179993,163.153061
2023-02-28 00:00:00-05:00,140.456635,15.980000,146.428741,145.214508,123.279999,98.679413,70.000000,259.365082,323.950012,179.317688,...,229.417175,29.987490,106.995491,61.146694,105.258034,101.084641,124.151634,122.413406,300.250000,164.641724
2023-03-31 00:00:00-04:00,136.865295,14.750000,163.802307,150.375778,124.400002,98.233154,67.870003,279.153290,385.369995,192.756485,...,228.311325,29.087189,110.490593,64.377197,105.018600,103.103394,128.955231,127.924393,318.000000,164.089630
2023-04-30 00:00:00-04:00,134.204605,13.640000,168.550491,143.903137,119.669998,107.703423,75.070000,274.836853,377.559998,175.808929,...,227.544983,28.874804,112.830528,66.735023,113.331215,102.256485,137.254120,137.073166,288.029999,173.669907
2023-05-31 00:00:00-04:00,114.623398,14.780000,176.313629,131.371597,109.769997,99.445549,69.699997,299.968231,417.790009,173.668518,...,215.018234,27.668062,97.669540,62.324833,98.704338,99.002373,126.231712,126.082764,262.570007,161.053207
2023-06-30 00:00:00-04:00,119.389526,17.940001,192.945312,128.295868,128.160004,106.289726,74.849998,302.576477,488.989990,191.313004,...,232.214905,32.566097,104.507416,59.839420,103.601883,111.273933,135.903397,144.402191,295.829987,170.142761
2023-07-31 00:00:00-04:00,120.898643,16.750000,195.412216,144.015747,152.190002,109.062447,77.690002,311.321106,546.169983,195.948273,...,208.382065,33.100613,107.842232,60.378433,103.592209,111.402382,135.040192,137.013458,307.959991,186.240829
2023-08-31 00:00:00-04:00,120.203651,14.730000,187.129883,141.493225,131.550003,100.804153,76.860001,318.623138,559.340027,178.516907,...,203.875824,32.011261,100.586861,54.988361,108.288322,102.633568,127.503113,118.140038,275.010010,188.637024
2023-09-30 00:00:00-04:00,111.019852,12.810000,170.535538,143.515106,137.210007,94.877380,79.709999,302.227966,509.899994,172.764343,...,206.866287,29.968403,91.687042,55.572105,114.511566,90.233086,123.127518,111.538483,236.529999,172.269547


In [162]:
def compute_log_returns(prices):
    """
    Compute log returns for each ticker.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    log_returns : DataFrame
        Log returns for each ticker and date
    """
    # TODO: Implement Function
    
    return np.log(prices) - np.log(prices.shift(1))

In [166]:
compute_log_returns(close)

Ticker,A,AAL,AAPL,ABBV,ABNB,ABT,ACGL,ACN,ADBE,ADI,...,WTW,WY,WYNN,XEL,XOM,XYL,YUM,ZBH,ZBRA,ZTS
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-03 00:00:00-05:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-01-04 00:00:00-05:00,0.010805,0.064588,0.010261,0.008035,0.044011,0.014765,0.004951,-0.003410,0.013239,0.021075,...,0.005329,0.011918,0.059958,0.008526,0.002906,-0.001536,0.013457,0.009228,0.010665,0.014266
2023-01-05 00:00:00-05:00,0.002897,0.029009,-0.010661,-0.001223,-0.011449,-0.003694,0.002705,-0.023897,-0.038730,-0.038211,...,-0.000203,-0.030231,0.004829,-0.020442,0.022127,-0.028804,-0.007645,-0.018701,-0.011204,-0.023845
2023-01-06 00:00:00-05:00,-0.029624,0.013490,0.036133,0.018544,0.009193,0.013714,0.015138,0.023414,0.013037,0.035858,...,0.027969,0.042326,0.024121,0.029177,0.012014,0.034934,0.020079,-0.003019,0.050097,0.014945
2023-01-09 00:00:00-05:00,-0.001355,0.029874,0.004081,-0.029800,0.008101,-0.001604,-0.019437,0.016723,0.027361,0.009501,...,0.006411,-0.006029,0.008090,0.009632,-0.018813,0.004663,-0.004076,0.002781,0.021242,-0.003936
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-12-22 00:00:00-05:00,0.004524,-0.002791,-0.005563,0.015283,-0.009120,0.005057,-0.004324,0.016212,-0.002319,0.004169,...,0.007274,0.007303,-0.003312,0.008577,0.001768,0.008470,0.000154,0.007577,0.015335,0.001642
2023-12-26 00:00:00-05:00,0.001718,-0.014075,-0.002845,-0.002068,-0.014883,0.001833,-0.004479,-0.002882,-0.000819,0.008940,...,-0.004174,0.007827,0.006394,0.003057,0.002254,0.008663,0.001998,0.009101,0.022353,0.002663
2023-12-27 00:00:00-05:00,0.000072,-0.008541,0.000518,0.001680,-0.015767,0.005114,0.000408,0.001103,-0.003651,0.002411,...,0.000878,0.003747,-0.004074,-0.003083,-0.004711,0.006667,0.004443,-0.002391,0.001052,0.007136


In [22]:
def get_top_n(prev_returns, top_n):
    """
    Select the top performing stocks
    
    Parameters
    ----------
    prev_returns : DataFrame
        Previous shifted returns for each ticker and date
    top_n : int
        The number of top performing stocks to get
    
    Returns
    -------
    top_stocks : DataFrame
        Top stocks for each ticker and date marked with a 1
    """
    # TODO: Implement Function

    stocks = pd.DataFrame(0, index=prev_returns.index, columns=prev_returns.columns)

    for date, returns in prev_returns.iterrows():
            top_tickers = returns.nlargest(top_n).index
            stocks.loc[date, top_tickers] = 1
    return stocks

In [24]:
top_bottom_n = 50
df_long = get_top_n(prev_returns, top_bottom_n)
df_short = get_top_n(-1*prev_returns, top_bottom_n)

NameError: name 'prev_returns' is not defined